# Breaking up Features

This notebook takes the individual JSONs each representing a single "feature" (IE wildfire) produced by the Step 1 Notebook and compiles all relevant fields and calculations into invidual CSV files so that they can later be used to produce visualizations.

The code is mostly adapted from the example code written for this assignment by David McDonald, licensed under the Creative Commons license. It also makes use of the Wildfire Reader module written by him for this assignment, and licensed under the same conditions.

To run the code, you will need the "USGS\_Wildland\_Fire\_Combined\_Dataset.json" that can be found within the "GeoJSON Files" zip archive available for download [here](https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81). Place the JSON file in the same folder as this notebook to run the code.

In [1]:
#This defines all of the packages and constants to be used in the notebook

import os, json, time
from pyproj import Transformer, Geod
from wildfire.Reader import Reader as WFReader
import geojson

EXTRACT_FILENAME = "USGS_Wildland_Fire_Combined_Dataset.json"
SAMPLE_DATA_FILENAME = EXTRACT_FILENAME

The below cell is copied almost verbatim from Dr. McDonald's example code, and is mostly here to test that the wildfire reader module was imported correctly and is able to see the JSON file.

In [2]:

#This bit of code opens a new wildfire reader, gets the header information and prints it to the screen
#
print(f"Attempting to open '{SAMPLE_DATA_FILENAME}' with wildfire.Reader() object")
wfreader = WFReader(SAMPLE_DATA_FILENAME)
print()
#
#    Now print the header - it contains some useful information
#
header_dict = wfreader.header()
header_keys = list(header_dict.keys())
print("The header has the following keys:")
print(header_keys)
print()
print("Header Dictionary")
print(json.dumps(header_dict,indent=4))

Attempting to open 'USGS_Wildland_Fire_Combined_Dataset.json' with wildfire.Reader() object

The header has the following keys:
['displayFieldName', 'fieldAliases', 'geometryType', 'spatialReference', 'fields']

Header Dictionary
{
    "displayFieldName": "",
    "fieldAliases": {
        "OBJECTID": "OBJECTID",
        "USGS_Assigned_ID": "USGS Assigned ID",
        "Assigned_Fire_Type": "Assigned Fire Type",
        "Fire_Year": "Fire Year",
        "Fire_Polygon_Tier": "Fire Polygon Tier",
        "Fire_Attribute_Tiers": "Fire Attribute Tiers",
        "GIS_Acres": "GIS_Acres",
        "GIS_Hectares": "GIS_Hectares",
        "Source_Datasets": "Source Datasets",
        "Listed_Fire_Types": "Listed Fire Types",
        "Listed_Fire_Names": "Listed Fire Names",
        "Listed_Fire_Codes": "Listed Fire Codes",
        "Listed_Fire_IDs": "Listed Fire IDs",
        "Listed_Fire_IRWIN_IDs": "Listed Fire IRWIN IDs",
        "Listed_Fire_Dates": "Listed Fire Dates",
        "Listed_Fire_C

The below cell is also copied almost verbatim from Dr. McDonald's example code, licensed under the aforementioned conditions, and is here to load the dataset into memory as a dict. The Max\_Feature\_Load constant was set to an arbitrarily high number so that every element in the 2GB JSON input file can be read.

In [3]:
#
#    This sample code will load the whole sample file, or a small amount of the complete dataset.
#
MAX_FEATURE_LOAD = 100000000
feature_list = list()
feature_count = 0
# A rewind() on the reader object makes sure we're at the start of the feature list
# This way, we can execute this cell multiple times and get the same result 
wfreader.rewind()
# Now, read through each of the features, saving them as dictionaries into a list
feature = wfreader.next()
while feature:
    feature_list.append(feature)
    feature_count += 1
    # if we're loading a lot of features, print progress
    if (feature_count % 5000) == 0:
        print(f"Loaded {feature_count} features")
    # loaded the max we're allowed then break
    if feature_count >= MAX_FEATURE_LOAD:
        break
    feature = wfreader.next()
#
#    Print the number of items (features) we think we loaded
print(f"Loaded a total of {feature_count} features")
#
#    Just a validation check - did all the items we loaded get into the list?
print(f"Variable 'feature_list' contains {len(feature_list)} features")




Loaded 5000 features


Loaded 10000 features


Loaded 15000 features


Loaded 20000 features


Loaded 25000 features


...

Loaded 115000 features


Loaded 120000 features


Loaded 125000 features


Loaded 130000 features


Loaded 135000 features
Loaded a total of 135061 features
Variable 'feature_list' contains 135061 features


TThe below cell is also copied exactly verbatim from Dr. McDonald's example code, licensed under the aforementioned conditions. It contains helper functions that take in a latitude-longitude point and ring data from a wildfire, and output either the distance between the closest part of the wildfire and that point, or the average distance between the entire wildfire and that point. These calculations will be stored in the CSVs made by this notebook for use in visualizations.

In [15]:
#
#    Transform feature geometry data
#
#    The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
#    The function returns a list of coordinates in EPSG:4326
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    #
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring

#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns a list containing the shortest distance to the perimeter and the point where that is
#
def shortest_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    closest_point = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        # convert the distance to miles
        distance_in_miles = d[2]*0.00062137
        # if it's closer to the city than the point we have, save it
        if not closest_point:
            closest_point.append(distance_in_miles)
            closest_point.append(point)
        elif closest_point and closest_point[0]>distance_in_miles:
            closest_point = list()
            closest_point.append(distance_in_miles)
            closest_point.append(point)
    return closest_point



#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns the average miles from boundary to the place
#
def average_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    # create a list to store our results
    distances_in_meters = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        distances_in_meters.append(d[2])
    #print("Got the following list:",distances_in_meters)
    # convert meters to miles
    distances_in_miles = [meters*0.00062137 for meters in distances_in_meters]
    # the esri polygon shape (the ring) requires that the first and last coordinates be identical to 'close the region
    # we remove one of them so that we don't bias our average by having two of the same point
    distances_in_miles_no_dup = distances_in_miles[1:]
    # now, average miles
    average = sum(distances_in_miles_no_dup)/len(distances_in_miles_no_dup)
    return average


The below cell is adapted from Dr. McDonald's example code, licensed under the aforementioned conditions, and stores all of the individual "features" as individual JSON files in the folder specified by the "destination\_folder" variable. Inside that folder, the JSONs will be organized into subfolders based on the year in which their wildfire happened, and then inside those based on whether or not the fires were in range of my target city (within 1250 miles of it). In practice, the code classified every wildfire as being in range, but this was compensated for in Notebook 3 to avoid having to re-run this cell, which is quite slow.

In [19]:
feature_subset = feature_list[0:13]
my_index = 0;
destination_folder = "feature_list"
for wf_feature in feature_list:
    if (my_index % 5000) == 0:
        print(f"Saved {my_index} features")
    
    wf_year = wf_feature['attributes']['Fire_Year']
    
    #I originally attempted to orgsnize the individual JSONs as "in range" and "out of range"
    #when compared to the location of my target city. Unfortinately, my code to determine 
    #the distance of each wildfire relative to my target city was bugged and labeled
    #every fire as in range. I accounted for this in later notebooks and successfully calculated
    #the distance relative to my target city for each wildfire in the notebook after this one. 
    #
    #I have commented out my distance code in this cell so that the code will run faster on repeat
    #runs. The lack of meaning in this variable is not a problem, due to being accounted for in later 
    #notebooks, as previously mentioned. 
    distance =  [0, 0] #shortest_distance_from_place_to_fire_perimeter(place['latlon'],ring_data)
    
    #print(distance)
    within_city = (distance[0] <= 1250)
    if within_city:
        city_string = "in_range"
    else:
        city_string = "not_within_range"
    year_folder = destination_folder + "/" + str(wf_year) + "/" + city_string + "/"
    if (not os.path.exists(year_folder)) :
        os.makedirs(year_folder)

    with open(year_folder + str(my_index) + ".json", "w") as outfile: 
        json.dump(wf_feature, outfile)

    my_index = my_index + 1

Saved 0 features


Saved 5000 features


Saved 10000 features


Saved 15000 features


Saved 20000 features


...

Saved 115000 features


Saved 120000 features


Saved 125000 features


Saved 130000 features


Saved 135000 features
